In [129]:
from faker import Faker
from datetime import timedelta
import pandas as pd
import numpy as np
import json
import os


## Inventory Data

In [130]:
inventory_df = pd.read_csv('./data/inventory_data.csv')
inventory_df

,Date,Item_ID,Item_Type,Item_Name,Current_Stock,Min_Required,Max_Capacity,Unit_Cost,Avg_Usage_Per_Day,Restock_Lead_Time,Vendor_ID
0,2024-10-01,105,Consumable,Ventilator,1542,264,1018,4467.55,108,17,V001
1,2024-10-02,100,Equipment,Ventilator,2487,656,3556,5832.29,55,12,V001
2,2024-10-03,103,Equipment,Surgical Mask,2371,384,5562,16062.98,470,6,V001
3,2024-10-04,103,Consumable,Surgical Mask,2038,438,1131,744.10,207,15,V002
4,2024-10-05,107,Equipment,IV Drip,2410,338,1013,15426.53,158,12,V003
...,...,...,...,...,...,...,...,...,...,...,...
495,2026-02-08,108,Consumable,Ventilator,4072,776,3496,15934.20,257,6,V002
496,2026-02-09,100,Equipment,X-ray Machine,997,239,2186,5863.88,108,16,V002
497,2026-02-10,107,Equipment,Surgical Mask,4193,304,4213,90.30,419,27,V001
498,2026-02-11,106,Consumable,IV Drip,822,186,2385,5284.45,382,6,V003


In [131]:
## First, generate unique items

supplies_dict = {
    'Ventilator': {
        'item_id': 100,
        'item_type': 'Equipment',
        'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
        'unit_cost': (1500000, 3200000),
        'min_required': (3, 15),
        'max_capacity': (25, 50),
        'avg_usage_per_day': (1, 5),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
        'storage_location': 'ICU Equipment Room',
    },
    'X-Ray-Machine': {
        'item_id': 101,
        'item_type': 'Equipment',
        'description': 'A diagnostic tool that uses radiation to create images of bones and internal structures.',
        'unit_cost': (2500000, 5000000),
        'min_required': (1, 5),
        'max_capacity': (10, 20),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 45),
        'expiration_days': None,
        'storage_location': 'Radiology Imaging Suite',
    },
    'Defibrilator': {
        'item_id': 102,
        'item_type': 'Equipment',
        'description': 'A life-saving device that delivers electric shocks to restore normal heart rhythms in cardiac arrest patients.',
        'unit_cost': (300000, 750000),
        'min_required': (1, 5),
        'max_capacity': (8, 15),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (20, 40),
        'expiration_days': None,
        'storage_location': 'Emergency Equipment Storage',
    },
    'Wheelchair': {
        'item_id': 103,
        'item_type': 'Equipment',
        'description': 'A mobility aid designed to assist patients with limited or no ability to walk.',
        'unit_cost': (5000, 25000), # Manual vs Electric Wheelchairs
        'min_required': (5, 20),
        'max_capacity': (30, 100),
        'avg_usage_per_day': (0, 5),
        'restock_lead_time': (7, 20),
        'expiration_days': None,
        'storage_location': 'Ward Equipment Room',
    },
    'Hospital Bed': {
        'item_id': 104,
        'item_type': 'Equipment',
        'description': 'An adjustable bed designed for patient care, enhancing comfort and accessibility in medical settings.',
        'unit_cost': (12000, 60000),
        'min_required': (25, 100),
        'max_capacity': (150, 300),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
        'storage_location': 'General Equipment Storage',
    },
    'ECG Machine': {
        'item_id': 105,
        'item_type': 'Equipment',
        'description': 'A device that records the heart’s electrical activity to detect irregularities and cardiac conditions.',
        'unit_cost': (100000, 400000),
        'min_required': (2, 5),
        'max_capacity': (10, 15),
        'avg_usage_per_day': (0, 3),
        'restock_lead_time': (15, 30),
        'expiration_days': None,
        'storage_location': 'Cardiology Equipment Room',
    },
    'MRI Scanner': {
        'item_id': 106,
        'item_type': 'Equipment',
        'description': 'An imaging system that uses magnetic fields and radio waves to create detailed images of internal organs.',
        'unit_cost': (25000000, 50000000),
        'min_required': (1, 2),
        'max_capacity': (2, 5),
        'avg_usage_per_day': (0, 1),
        'restock_lead_time': (30, 60),
        'expiration_days': None,
        'storage_location': 'Radiology Imaging Suite',
    },
    'Infusion Pump': {
        'item_id': 107,
        'item_type': 'Equipment',
        'description': 'A device that precisely delivers fluids, medications, or nutrients into a patient’s bloodstream.',
        'unit_cost': (80000, 250000),
        'min_required': (3, 10),
        'max_capacity': (15, 30),
        'avg_usage_per_day': (5, 15),
        'restock_lead_time': (10, 20),
        'expiration_days': None,
        'storage_location': 'ICU Equipment Room',
    },
    'Gloves': {
        'item_id': 200,
        'item_type': 'Consumable',
        'description': 'Protective hand coverings used by medical staff to maintain hygiene and prevent contamination.',
        'unit_cost': (5, 15),
        'min_required': (1000, 3000),
        'max_capacity': (5000, 10000),
        'avg_usage_per_day': (200, 1000),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'IV Drip': {
        'item_id': 201,
        'item_type': 'Consumable',
        'description': 'A method of delivering fluids, nutrients, or medications directly into a patient\'s veins.',
        'unit_cost': (120, 350),
        'min_required': (300, 1000),
        'max_capacity': (2000, 5000),
        'avg_usage_per_day': (50, 200),
        'restock_lead_time': (5, 10),
        'expiration_days': 1825,
        'storage_location': 'Pharmacy Storage',
    },
    'Surgical Mask': {
        'item_id': 202,
        'item_type': 'Consumable',
        'description': 'A disposable face mask that protects against airborne contaminants and reduces infection spread.',
        'unit_cost': (3, 8),
        'min_required': (3000, 10000),
        'max_capacity': (15000, 50000),
        'avg_usage_per_day': (500, 3000),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Gown': {
        'item_id': 203,
        'item_type': 'Consumable',
        'description': 'A protective garment worn by medical staff to prevent contamination and ensure a sterile environment.',
        'unit_cost': (80, 250),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Face Shield': {
        'item_id': 204,
        'item_type': 'Consumable',
        'description': 'A transparent protective visor that guards against splashes, droplets, and airborne particles.',
        'unit_cost': (50, 120),
        'min_required': (500, 2000),
        'max_capacity': (4000, 10000),
        'avg_usage_per_day': (100, 800),
        'restock_lead_time': (3, 8),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Syringe': {
        'item_id': 205,
        'item_type': 'Consumable',
        'description': 'A medical instrument used to inject or withdraw fluids from the body.',
        'unit_cost': (7, 35),
        'min_required': (1500, 5000),
        'max_capacity': (8000, 20000),
        'avg_usage_per_day': (200, 2000),
        'restock_lead_time': (5, 12),
        'expiration_days': 1825,
        'storage_location': 'Pharmacy Storage',
    },
    'Bandages': {
        'item_id': 206,
        'item_type': 'Consumable',
        'description': 'Sterile materials used to cover wounds and promote healing while preventing infections.',
        'unit_cost': (10, 50),
        'min_required': (1000, 4000),
        'max_capacity': (6000, 12000),
        'avg_usage_per_day': (300, 1500),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
    'Blood Bags': {
        'item_id': 207,
        'item_type': 'Consumable',
        'description': 'Special containers designed for the collection, storage, and transfusion of blood.',
        'unit_cost': (400, 1500),
        'min_required': (300, 1000),
        'max_capacity': (1500, 5000),
        'avg_usage_per_day': (20, 150),
        'restock_lead_time': (5, 15),
        'expiration_days': 42,
        'storage_location': 'Emergency Consumables Depot',
    },
    'Antiseptic Solution': {
        'item_id': 208,
        'item_type': 'Consumable',
        'description': 'A liquid disinfectant used to cleanse wounds and prevent infections.',
        'unit_cost': (200, 700),
        'min_required': (500, 2000),
        'max_capacity': (3000, 8000),
        'avg_usage_per_day': (100, 500),
        'restock_lead_time': (5, 12),
        'expiration_days': 1095,
        'storage_location': 'Emergency Consumables Depot',
    },
    'Cotton Rolls': {
        'item_id': 209,
        'item_type': 'Consumable',
        'description': 'Soft, absorbent materials used in medical applications for wound care and cleaning.',
        'unit_cost': (50, 150),
        'min_required': (300, 1000),
        'max_capacity': (2000, 6000),
        'avg_usage_per_day': (50, 300),
        'restock_lead_time': (3, 10),
        'expiration_days': 1825,
        'storage_location': 'Medical Supply Room',
    },
}

In [132]:
item_var_dict = {}
fake = Faker()
supplier_names = [fake.company() + ' Medical Supplies' for _ in range(5)]
contact_numbers = [fake.numerify("+63 9## ### ####") for _ in range(5)]

for key, value in supplies_dict.items():
    # Generate 5 different item variations for 5 different vendors
    item_name = key
    item_id = value['item_id']
    item_type = value['item_type']
    description = value['description']
    status = 'Active'
    min_required = value['min_required'][0]
    max_capacity = value['max_capacity'][0]

    for i in range(5):
        vendor_id = f'V{str(i+1).zfill(3)}'
        vendor_name = supplier_names[i]
        vendor_contact = contact_numbers[i]

        item_var_dict[f'{item_name}-{vendor_id}'] = {
            'item_name': item_name,
            'item_id': item_id,
            'item_type': item_type,
            'description': description,
            'status': status,
            'min_required': min_required,
            'max_capacity': max_capacity,
            'vendor_name': vendor_name,
            'vendor_contact': vendor_contact,
        }

item_var_dict

{'Ventilator-V001': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 25,
  'vendor_name': 'Good-Stewart Medical Supplies',
  'vendor_contact': '+63 944 418 9967'},
 'Ventilator-V002': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_capacity': 25,
  'vendor_name': 'Stanton, Woodard and Webb Medical Supplies',
  'vendor_contact': '+63 952 549 4695'},
 'Ventilator-V003': {'item_name': 'Ventilator',
  'item_id': 100,
  'item_type': 'Equipment',
  'description': 'A medical device that supports or takes over breathing for patients with respiratory failure.',
  'status': 'Active',
  'min_required': 3,
  'max_c

In [133]:
fake = Faker()
date_list = pd.date_range(start='2022-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

# Data dictionary (to pass to df constructor)
date_arr = []
item_id_arr = []
item_name_arr = []
item_type_arr = []
description_arr = []
status_arr = []
stock_status_arr = []
unit_cost_arr = []
min_required_arr = []
max_capacity_arr = []
current_stock_arr = []
avg_usage_per_day_arr = []
restock_lead_time_arr = []
vendor_name_arr = []
vendor_contact_arr = []
expiration_date_arr = []
created_by_arr = []

trend_factor = np.ones(len(date_list))
noise_factor = np.random.normal(1.0, 0.05, len(date_list))

def seasonal_multiplier(day_of_year):
    return 1 + 0.05 * np.sin(2 * np.pi * day_of_year / 365)  # Weak seasonality

def accept_unused_date(used_dates_set):
    while True:
        selected_date = fake.random_element(date_list)
        if selected_date not in used_dates_set:
            used_dates_set.add(selected_date)
            return selected_date

current_unique_item = ''
used_dates = set()

for key, value in item_var_dict.items():
    if current_unique_item != value['item_name']:
        current_unique_item = value['item_name']
        used_dates = set()

    # Generate around 200 - 215 random instances of a particular item
    for _ in range(np.random.randint(200, 216)):
        date = accept_unused_date(used_dates)
        month = pd.to_datetime(date).month
        date_obj = pd.to_datetime(date)
        day_of_year = date_obj.dayofyear
        time_index = date_list.index(date)
        
        item_id = value['item_id']
        item_name = value['item_name']
        item_type = value['item_type']
        description = value['description']
        status = value['status']
        stock_status = 'Available'
        unit_cost = np.random.randint(supplies_dict[item_name]['unit_cost'][0], supplies_dict[item_name]['unit_cost'][1] + 1)
        min_required = value['min_required']
        max_capacity = value['max_capacity']

        base_stock = np.random.randint(value['min_required'], value['max_capacity'] + 1)
        stock_with_trend = base_stock * trend_factor[time_index]  # No trend
        stock_seasonal = stock_with_trend * seasonal_multiplier(day_of_year)  # No seasonality
        stock_randomized = stock_seasonal * noise_factor[time_index]  # Small random noise
        current_stock = int(np.clip(stock_randomized, value['min_required'], value['max_capacity']))  # Keep within limits

        avg_usage_per_day = np.random.randint(supplies_dict[item_name]['avg_usage_per_day'][0], supplies_dict[item_name]['avg_usage_per_day'][1] + 1)
        restock_lead_time = np.random.randint(supplies_dict[item_name]['restock_lead_time'][0], supplies_dict[item_name]['restock_lead_time'][1] + 1)
        vendor_name = value['vendor_name']
        vendor_contact = value['vendor_contact']
        created_by = '67b1fdb293bfc8e17f896b31'

        if item_type == 'Consumable':
            expiration_duration = supplies_dict[item_name]['expiration_days']
            expiration_date = pd.to_datetime(date) + timedelta(days=expiration_duration)
        else:
            expiration_date = pd.NaT

        date_arr.append(date)
        item_id_arr.append(item_id)
        item_name_arr.append(item_name)
        item_type_arr.append(item_type)
        description_arr.append(description)
        status_arr.append(status)
        stock_status_arr.append(stock_status)
        unit_cost_arr.append(unit_cost)
        min_required_arr.append(min_required)
        max_capacity_arr.append(max_capacity)
        current_stock_arr.append(current_stock)
        avg_usage_per_day_arr.append(avg_usage_per_day)
        restock_lead_time_arr.append(restock_lead_time)
        vendor_name_arr.append(vendor_name)
        vendor_contact_arr.append(vendor_contact)
        expiration_date_arr.append(expiration_date)
        created_by_arr.append(created_by)

data = list(zip(date_arr, item_id_arr, item_name_arr, item_type_arr, description_arr, status_arr, stock_status_arr, unit_cost_arr, min_required_arr, max_capacity_arr, 
                current_stock_arr, avg_usage_per_day_arr, restock_lead_time_arr, vendor_name_arr, vendor_contact_arr, expiration_date_arr, created_by_arr))

df = pd.DataFrame(data, columns=[
    'date', 'item_id', 'item_name', 'item_type', 'description', 'status', 'stock_status', 'unit_cost', 'min_required', 'max_capacity', 'current_stock', 'avg_usage_per_day', 'restock_lead_time', 'vendor_name', 'vendor_contact', 'expiration_date', 'created_by'
]).sort_values(by=['date']).reset_index(drop=True)

## Item Listing Data

In [134]:
item_listing_df = df[['item_id', 'item_name', 'description', 'item_type', 'min_required', 'max_capacity', 'created_by', 'status']].drop_duplicates().sort_values('item_id').reset_index(drop=True)
item_listing_df

,item_id,item_name,description,item_type,min_required,max_capacity,created_by,status
0,100,Ventilator,A medical device that supports or takes over b...,Equipment,3,25,67b1fdb293bfc8e17f896b31,Active
1,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,1,10,67b1fdb293bfc8e17f896b31,Active
2,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,1,8,67b1fdb293bfc8e17f896b31,Active
3,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,5,30,67b1fdb293bfc8e17f896b31,Active
4,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,25,150,67b1fdb293bfc8e17f896b31,Active
5,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,2,10,67b1fdb293bfc8e17f896b31,Active
6,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,1,2,67b1fdb293bfc8e17f896b31,Active
7,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,3,15,67b1fdb293bfc8e17f896b31,Active
8,200,Gloves,Protective hand coverings used by medical staf...,Consumable,1000,5000,67b1fdb293bfc8e17f896b31,Active
9,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,300,2000,67b1fdb293bfc8e17f896b31,Active


In [135]:
file_name_csv = './data/item_listing_v1.csv'
file_name_json = './data/item_listing_v1.json'

if not os.path.exists(file_name_csv):
    item_listing_df.to_csv(file_name_csv, index=False)
    item_listing_df.to_json(file_name_json, orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!


## Item Stock Listing Data

In [136]:
item_listing_df = pd.read_json('./data/exported_listings.json')
item_listing_df

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67b5d1d3965831513a85d8be,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67b5d1d3965831513a85d8bf,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67b5d1d3965831513a85d8c0,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67b5d1d3965831513a85d8c1,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67b5d1d3965831513a85d8c2,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
5,67b5d1d3965831513a85d8c3,105,ECG Machine,A device that records the heart’s electrical a...,Equipment,C,2,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
6,67b5d1d3965831513a85d8c4,106,MRI Scanner,An imaging system that uses magnetic fields an...,Equipment,C,1,2,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
7,67b5d1d3965831513a85d8c5,107,Infusion Pump,"A device that precisely delivers fluids, medic...",Equipment,C,3,15,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
8,67b5d1d3965831513a85d8c6,200,Gloves,Protective hand coverings used by medical staf...,Consumable,C,1000,5000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
9,67b5d1d3965831513a85d8c7,201,IV Drip,"A method of delivering fluids, nutrients, or m...",Consumable,C,300,2000,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [137]:
df

,date,item_id,item_name,item_type,description,status,stock_status,unit_cost,min_required,max_capacity,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,created_by
0,2022-01-01,206,Bandages,Consumable,Sterile materials used to cover wounds and pro...,Active,Available,49,1000,6000,1576,1413,3,Robles-Price Medical Supplies,+63 960 400 1304,2026-12-31,67b1fdb293bfc8e17f896b31
1,2022-01-01,105,ECG Machine,Equipment,A device that records the heart’s electrical a...,Active,Available,324548,2,10,7,2,15,Lopez Group Medical Supplies,+63 980 536 9299,NaT,67b1fdb293bfc8e17f896b31
2,2022-01-01,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,12,1000,5000,4971,610,7,"Miller, Wilson and Hunt Medical Supplies",+63 985 663 4894,2026-12-31,67b1fdb293bfc8e17f896b31
3,2022-01-01,102,Defibrilator,Equipment,A life-saving device that delivers electric sh...,Active,Available,330189,1,8,8,1,33,Lopez Group Medical Supplies,+63 980 536 9299,NaT,67b1fdb293bfc8e17f896b31
4,2022-01-01,205,Syringe,Consumable,A medical instrument used to inject or withdra...,Active,Available,8,1500,8000,4602,566,9,"Stanton, Woodard and Webb Medical Supplies",+63 952 549 4695,2026-12-31,67b1fdb293bfc8e17f896b31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18638,2024-12-31,206,Bandages,Consumable,Sterile materials used to cover wounds and pro...,Active,Available,21,1000,6000,2422,729,9,Lopez Group Medical Supplies,+63 980 536 9299,2029-12-30,67b1fdb293bfc8e17f896b31
18639,2024-12-31,209,Cotton Rolls,Consumable,"Soft, absorbent materials used in medical appl...",Active,Available,124,300,2000,1219,92,4,"Miller, Wilson and Hunt Medical Supplies",+63 985 663 4894,2029-12-30,67b1fdb293bfc8e17f896b31
18640,2024-12-31,204,Face Shield,Consumable,A transparent protective visor that guards aga...,Active,Available,106,500,4000,818,295,3,"Stanton, Woodard and Webb Medical Supplies",+63 952 549 4695,2029-12-30,67b1fdb293bfc8e17f896b31
18641,2024-12-31,102,Defibrilator,Equipment,A life-saving device that delivers electric sh...,Active,Available,411516,1,8,7,1,34,Lopez Group Medical Supplies,+63 980 536 9299,NaT,67b1fdb293bfc8e17f896b31


In [138]:
df[(df['item_name'] == 'Gloves') & (df['date'].str.startswith('2024-01'))]

,date,item_id,item_name,item_type,description,status,stock_status,unit_cost,min_required,max_capacity,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,created_by
12437,2024-01-01,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,14,1000,5000,1844,338,5,Robles-Price Medical Supplies,+63 960 400 1304,2028-12-30,67b1fdb293bfc8e17f896b31
12445,2024-01-02,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,15,1000,5000,2478,339,11,Lopez Group Medical Supplies,+63 980 536 9299,2028-12-31,67b1fdb293bfc8e17f896b31
12469,2024-01-03,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,6,1000,5000,2296,261,6,Lopez Group Medical Supplies,+63 980 536 9299,2029-01-01,67b1fdb293bfc8e17f896b31
12489,2024-01-04,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,9,1000,5000,1458,614,10,"Miller, Wilson and Hunt Medical Supplies",+63 985 663 4894,2029-01-02,67b1fdb293bfc8e17f896b31
12505,2024-01-05,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,10,1000,5000,4540,282,6,"Stanton, Woodard and Webb Medical Supplies",+63 952 549 4695,2029-01-03,67b1fdb293bfc8e17f896b31
12513,2024-01-06,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,7,1000,5000,3710,692,11,"Stanton, Woodard and Webb Medical Supplies",+63 952 549 4695,2029-01-04,67b1fdb293bfc8e17f896b31
12540,2024-01-07,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,10,1000,5000,2236,971,6,Robles-Price Medical Supplies,+63 960 400 1304,2029-01-05,67b1fdb293bfc8e17f896b31
12550,2024-01-08,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,9,1000,5000,3794,609,11,Robles-Price Medical Supplies,+63 960 400 1304,2029-01-06,67b1fdb293bfc8e17f896b31
12591,2024-01-10,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,10,1000,5000,4684,383,7,"Stanton, Woodard and Webb Medical Supplies",+63 952 549 4695,2029-01-08,67b1fdb293bfc8e17f896b31
12612,2024-01-11,200,Gloves,Consumable,Protective hand coverings used by medical staf...,Active,Available,8,1000,5000,3925,255,10,"Miller, Wilson and Hunt Medical Supplies",+63 985 663 4894,2029-01-09,67b1fdb293bfc8e17f896b31


In [139]:
df_sub = df[['item_id', 'date', 'stock_status', 'unit_cost', 'current_stock', 'avg_usage_per_day', 'restock_lead_time', 'vendor_name', 'vendor_contact', 'expiration_date']]

stock_listing_df = item_listing_df[['_id', 'itemCode']].merge(df_sub, left_on='itemCode', right_on='item_id')
stock_listing_df.drop(columns=['itemCode', 'item_id'], inplace=True)
stock_listing_df.sort_values(by='date', inplace=True)
stock_listing_df

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2955430,15,1,16,Lopez Group Medical Supplies,+63 980 536 9299,NaT
3086,67b5d1d3965831513a85d8c1,2022-01-01,Available,17355,17,1,11,Robles-Price Medical Supplies,+63 960 400 1304,NaT
17612,67b5d1d3965831513a85d8cf,2022-01-01,Available,136,1427,238,6,Lopez Group Medical Supplies,+63 980 536 9299,2026-12-31
4128,67b5d1d3965831513a85d8c2,2022-01-01,Available,35360,59,1,16,"Miller, Wilson and Hunt Medical Supplies",+63 985 663 4894,NaT
16589,67b5d1d3965831513a85d8ce,2022-01-01,Available,385,2045,413,5,Robles-Price Medical Supplies,+63 960 400 1304,2024-12-31
...,...,...,...,...,...,...,...,...,...,...
3085,67b5d1d3965831513a85d8c0,2024-12-31,Available,411516,7,1,34,Lopez Group Medical Supplies,+63 980 536 9299,NaT
2060,67b5d1d3965831513a85d8bf,2024-12-31,Available,2725453,5,1,40,Lopez Group Medical Supplies,+63 980 536 9299,NaT
1023,67b5d1d3965831513a85d8be,2024-12-31,Available,2753755,16,4,18,Robles-Price Medical Supplies,+63 960 400 1304,NaT
17611,67b5d1d3965831513a85d8ce,2024-12-31,Available,388,1913,397,8,Lopez Group Medical Supplies,+63 980 536 9299,2027-12-31


In [140]:
file_name_json = './data/stock_listing_v1_1.json'

if not os.path.exists(file_name_json):
    stock_listing_df.to_json(file_name_json, orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!


## Patient Data

In [141]:
patient_df = pd.read_csv('./data/patient_data.csv')
patient_df

,Patient_ID,Admission_Date,Discharge_Date,Primary_Diagnosis,Procedure_Performed,Room_Type,Bed_Days,Supplies_Used,Equipment_Used,Staff_Needed
0,P001,2024-10-06 05:30:28,2024-10-23 01:11:34,Diabetes,Appendectomy,General Ward,2,"Gloves, IV",Surgical Table,2 Surgeons
1,P002,2024-10-24 11:07:58,2024-10-15 05:16:54,Fracture,Appendectomy,ICU,10,"Gown, IV",MRI Machine,1 Nurse
2,P003,2024-10-22 21:43:43,2024-10-24 10:56:30,Fracture,Chest X-ray,ICU,10,"Gloves, IV",X-ray Machine,"1 Nurse, 1 Doctor"
3,P004,2024-10-05 17:04:05,2024-10-30 14:10:01,Diabetes,Chest X-ray,ICU,11,"Gloves, IV",X-ray Machine,1 Nurse
4,P005,2024-10-21 17:04:00,2024-10-08 15:53:22,Appendicitis,MRI,ICU,2,"Gloves, IV",X-ray Machine,2 Surgeons
...,...,...,...,...,...,...,...,...,...,...
495,P496,2024-10-01 12:54:33,2024-10-06 14:31:39,Diabetes,MRI,ICU,13,"Mask, Gown",Surgical Table,"1 Nurse, 1 Doctor"
496,P497,2024-10-22 07:37:10,2024-10-13 12:31:40,Appendicitis,Appendectomy,ICU,14,"Gloves, IV",MRI Machine,"1 Nurse, 1 Doctor"
497,P498,2024-10-28 03:12:20,2024-10-10 08:11:18,Pneumonia,MRI,General Ward,13,"Mask, Gown",X-ray Machine,2 Surgeons
498,P499,2024-10-04 22:02:01,2024-10-12 03:45:09,Diabetes,Chest X-ray,General Ward,1,"Gown, IV",Surgical Table,"1 Nurse, 1 Doctor"


In [142]:
# Supplies Used Item Mapping
patient_supplies_dict = {}

for i in patient_df['Supplies_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in patient_supplies_dict:
            patient_supplies_dict[item] = 1
        elif item in patient_supplies_dict:
            patient_supplies_dict[item] += 1
        
patient_supplies_dict

{'Gloves': 169, 'IV': 339, 'Gown': 331, 'Mask': 161}

In [143]:
# Equipment Used Item Mapping
equipment_dict = {}

for i in patient_df['Equipment_Used']:
    item_arr = i.split(', ')
    for item in item_arr:
        if item not in equipment_dict:
            equipment_dict[item] = 1
        elif item in equipment_dict:
            equipment_dict[item] += 1
        
equipment_dict

{'Surgical Table': 166, 'MRI Machine': 161, 'X-ray Machine': 173}

In [144]:
# Rename the keys in the dictionary to standardized item names
try:
    patient_supplies_dict['IV Drip'] = patient_supplies_dict.pop('IV')
    patient_supplies_dict['Surgical Mask'] = patient_supplies_dict.pop('Mask')
    equipment_dict['X-Ray-Machine'] = equipment_dict.pop('X-ray Machine')
    equipment_dict['MRI Scanner'] = equipment_dict.pop('MRI Machine')
except KeyError:
    pass

In [145]:
# EDA of 'primary_diagnosis'
patient_df['Primary_Diagnosis'].value_counts()

Primary_Diagnosis
Fracture        133
Diabetes        131
Appendicitis    120
Pneumonia       116
Name: count, dtype: int64

In [146]:
# EDA of 'procedure_performed'
patient_df['Procedure_Performed'].value_counts()

Procedure_Performed
Chest X-ray     138
Appendectomy    131
MRI             131
Blood Test      100
Name: count, dtype: int64

### Synthetic Patient Data Generation

In [147]:
diagnosis_dict = {
    'Pneumonia': {
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        },
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        }
    },
    'Appendicitis': {
        'Appendectomy': {
            'supplies_used': ['Gloves', 'Gown', 'Surgical Mask', 'Antiseptic Solution', 'Bandages', 'IV Drip'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        }
    },
    'Fracture': {
        'X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        },
        'Casting': {
            'supplies_used': ['Bandages', 'Cotton Rolls', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': []
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'Myocardial Infarction': {
        'ECG': {
            'supplies_used': ['Gloves'],
            'equipment_used': ['ECG Machine']
        },
        'Defibrillation': {
            'supplies_used': ['Gloves', 'Surgical Mask', 'Antiseptic Solution'],
            'equipment_used': ['Defibrillator']
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Diabetes Complications': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'IV Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves', 'Blood Bags'],
            'equipment_used': ['Infusion Pump']
        },
        'Wound Dressing': {
            'supplies_used': ['Bandages', 'Gloves', 'Antiseptic Solution', 'Cotton Rolls'],
            'equipment_used': []
        }
    },
    'Stroke': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Physical Therapy': {
            'supplies_used': ['Gloves', 'Face Shield', 'Gown'],
            'equipment_used': ['Hospital Bed', 'Wheelchair']
        }
    },
    'Sepsis': {
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves', 'Blood Bags'],
            'equipment_used': []
        },
        'IV Antibiotic Therapy': {
            'supplies_used': ['IV Drip', 'Gloves', 'Antiseptic Solution'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Severe Burns': {
        'Wound Cleaning & Dressing': {
            'supplies_used': ['Antiseptic Solution', 'Bandages', 'Gloves', 'Cotton Rolls'],
            'equipment_used': []
        },
        'Skin Grafting Surgery': {
            'supplies_used': ['Gown', 'Surgical Mask', 'Gloves'],
            'equipment_used': ['Hospital Bed']
        },
        'IV Fluid Therapy': {
            'supplies_used': ['IV Drip', 'Syringe', 'Gloves'],
            'equipment_used': ['Infusion Pump']
        }
    },
    'Traumatic Brain Injury': {
        'MRI': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['MRI Scanner']
        },
        'Ventilation Support': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves'],
            'equipment_used': ['Ventilator']
        },
        'Pain Management': {
            'supplies_used': ['Syringe', 'IV Drip', 'Gloves'],
            'equipment_used': ['Hospital Bed', 'Infusion Pump']
        }
    },
    'COVID-19': {
        'Oxygen Therapy': {
            'supplies_used': ['Surgical Mask', 'Face Shield', 'Gloves', 'Gown'],
            'equipment_used': ['Ventilator', 'Hospital Bed']
        },
        'Blood Test': {
            'supplies_used': ['Syringe', 'Cotton Rolls', 'Gloves'],
            'equipment_used': []
        },
        'Chest X-Ray': {
            'supplies_used': ['Surgical Mask', 'Gloves', 'Gown', 'Face Shield'],
            'equipment_used': ['X-Ray Machine']
        }
    }
}

In [148]:
diagnosis_id_arr = []
date_arr = []
primary_diagnosis_arr = []
procedure_performed_arr = []
item_used_arr = []
item_type_arr = []

diagnosis_id_counter = 1

date_list = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D').strftime('%Y-%m-%d').tolist()

for diagnosis in diagnosis_dict:
    for _ in range(np.random.randint(100, 500)):
        diagnosis_id = diagnosis_id_counter
        for procedure in diagnosis_dict[diagnosis]:
            for item in diagnosis_dict[diagnosis][procedure]['supplies_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Consumable')
            for item in diagnosis_dict[diagnosis][procedure]['equipment_used']:
                diagnosis_id_arr.append(str(diagnosis_id).zfill(5))
                date_arr.append(fake.random_element(date_list))
                primary_diagnosis_arr.append(diagnosis)
                procedure_performed_arr.append(procedure)
                item_used_arr.append(item)
                item_type_arr.append('Equipment')
        diagnosis_id_counter += 1

In [149]:
print(len(diagnosis_id_arr))
print(len(primary_diagnosis_arr))
print(len(procedure_performed_arr))
print(len(item_used_arr))
print(len(item_type_arr))

31037
31037
31037
31037
31037


In [150]:
# Data dictionary (to pass to df constructor)
data = {
    'diagnosis_id': diagnosis_id_arr,
    'date': date_arr,
    'primary_diagnosis': primary_diagnosis_arr,
    'procedure_performed': procedure_performed_arr,
    'item_used': item_used_arr,
    'item_type': item_type_arr
}

patient_df_v2 = pd.DataFrame(data)
patient_df_v2

,diagnosis_id,date,primary_diagnosis,procedure_performed,item_used,item_type
0,00001,2024-01-20,Pneumonia,Chest X-Ray,Surgical Mask,Consumable
1,00001,2024-06-10,Pneumonia,Chest X-Ray,Gloves,Consumable
2,00001,2024-07-15,Pneumonia,Chest X-Ray,Gown,Consumable
3,00001,2024-08-03,Pneumonia,Chest X-Ray,Face Shield,Consumable
4,00001,2024-06-14,Pneumonia,Chest X-Ray,X-Ray Machine,Equipment
...,...,...,...,...,...,...
31032,02585,2024-06-21,COVID-19,Chest X-Ray,Surgical Mask,Consumable
31033,02585,2024-09-13,COVID-19,Chest X-Ray,Gloves,Consumable
31034,02585,2024-05-08,COVID-19,Chest X-Ray,Gown,Consumable
31035,02585,2024-08-07,COVID-19,Chest X-Ray,Face Shield,Consumable


In [151]:
file_name = './data/patient_data_v2.csv'

if not os.path.exists(file_name):
    patient_df_v2.to_csv(file_name, index=False)
    print('File saved successfully!')
else:
    print('File already exists!')

File already exists!


# Stock Listing V2
Add 'storageLocation' column to data

In [152]:
stock_listing_df1 = pd.read_json('./data/stock_listing_v1_1.json')
stock_listing_df1 = stock_listing_df1.merge(item_listing_df[['_id', 'title']], on='_id')
stock_listing_df1.head()

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,title
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,26,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,Ventilator
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,29,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Defibrilator
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,17,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Wheelchair
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,18,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,ECG Machine
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,37,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,MRI Scanner


In [153]:
# Map defined storageLocation to each stock listing record
stock_listing_df1 = stock_listing_df1.copy()
stock_listing_df1['storageLocation'] = stock_listing_df1['title'].map(lambda x: supplies_dict[x]['storage_location'])
stock_listing_df1

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,restock_lead_time,vendor_name,vendor_contact,expiration_date,title,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,26,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,Ventilator,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,29,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Defibrilator,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,17,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Wheelchair,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,18,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,ECG Machine,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,37,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,MRI Scanner,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,11,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Syringe,Pharmacy Storage
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,28,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ventilator,ICU Equipment Room
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,9,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Bandages,Medical Supply Room
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,5,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Gloves,Medical Supply Room


In [154]:
# Remove title column
stock_listing_df1 = stock_listing_df1.drop(['title', 'restock_lead_time'], axis=1)
stock_listing_df1

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,vendor_name,vendor_contact,expiration_date,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [155]:
# Export stock_listing_v2_1.json (used for model training only!)
if not os.path.exists('./data/stock_listing_v2_1.json'):
    stock_listing_df1.to_json('./data/stock_listing_v2_1.json', orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')


File already exists!


In [156]:
# Rename ALL columns from snake case to camel case
stock_listing_df1 = stock_listing_df1.rename(columns={
    'date': 'acquisitionDate',
    '_id': 'listing',
    'current_stock': 'quantity',
    'avg_usage_per_day': 'avgUsagePerDay',
    'stock_status': 'status',
    'unit_cost': 'unitCost',
    'vendor_name': 'supplierName',
    'vendor_contact': 'supplierContact',
    'expiration_date': 'expirationDate',
})
stock_listing_df1.head(5)

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite


In [157]:
# Export stock_listing_v2_2.json (to export to database)
if not os.path.exists('./data/stock_listing_v2_2.json'):
    stock_listing_df1.to_json('./data/stock_listing_v2_2.json', orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')


File already exists!


## Update Item Listing IDs in Stock Listing

In [158]:
stock_listing_df3 = pd.read_json('./data/stock_listing_v2_1.json')
stock_listing_df3

,_id,date,stock_status,unit_cost,current_stock,avg_usage_per_day,vendor_name,vendor_contact,expiration_date,storageLocation
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [159]:
# Rename ALL columns from snake case to camel case
stock_listing_df3 = stock_listing_df3.rename(columns={
    'date': 'acquisitionDate',
    '_id': 'listing',
    'current_stock': 'quantity',
    'avg_usage_per_day': 'avgUsagePerDay',
    'stock_status': 'status',
    'unit_cost': 'unitCost',
    'vendor_name': 'supplierName',
    'vendor_contact': 'supplierContact',
    'expiration_date': 'expirationDate',
    }
)

In [160]:
old_item_listing = pd.read_json('./data/exported_listings.json')
old_item_listing.head()

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67b5d1d3965831513a85d8be,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67b5d1d3965831513a85d8bf,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67b5d1d3965831513a85d8c0,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67b5d1d3965831513a85d8c1,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67b5d1d3965831513a85d8c2,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [161]:
stock_listing_df3 = stock_listing_df3.merge(old_item_listing[['_id', 'title']], left_on='listing', right_on='_id')
stock_listing_df3.drop('_id', axis=1, inplace=True)
stock_listing_df3

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation,title
0,67b5d1d3965831513a85d8be,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room,Ventilator
1,67b5d1d3965831513a85d8c0,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage,Defibrilator
2,67b5d1d3965831513a85d8c1,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room,Wheelchair
3,67b5d1d3965831513a85d8c3,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room,ECG Machine
4,67b5d1d3965831513a85d8c4,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite,MRI Scanner
...,...,...,...,...,...,...,...,...,...,...,...
18643,67b5d1d3965831513a85d8cb,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage,Syringe
18644,67b5d1d3965831513a85d8be,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room,Ventilator
18645,67b5d1d3965831513a85d8cc,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room,Bandages
18646,67b5d1d3965831513a85d8c6,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room,Gloves


In [162]:
new_item_listing = pd.read_json('./data/exported_listings_v3.json')
new_item_listing.head()

,_id,itemCode,title,description,category,abcCategory,minStockLevel,maxStockLevel,createdBy,status,createdAt
0,67be98a8ea3e2aaa233b8709,100,Ventilator,A medical device that supports or takes over b...,Equipment,C,3,25,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
1,67be98a8ea3e2aaa233b870a,101,X-Ray-Machine,A diagnostic tool that uses radiation to creat...,Equipment,C,1,10,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
2,67be98a8ea3e2aaa233b870b,102,Defibrilator,A life-saving device that delivers electric sh...,Equipment,C,1,8,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
3,67be98a8ea3e2aaa233b870c,103,Wheelchair,A mobility aid designed to assist patients wit...,Equipment,C,5,30,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z
4,67be98a8ea3e2aaa233b870d,104,Hospital Bed,"An adjustable bed designed for patient care, e...",Equipment,C,25,150,67b1fdb293bfc8e17f896b31,active,2025-02-19T12:42:59.839Z


In [163]:
new_item_map = new_item_listing.set_index('title')['_id'].to_dict()
new_item_map

{'Ventilator': '67be98a8ea3e2aaa233b8709',
 'X-Ray-Machine': '67be98a8ea3e2aaa233b870a',
 'Defibrilator': '67be98a8ea3e2aaa233b870b',
 'Wheelchair': '67be98a8ea3e2aaa233b870c',
 'Hospital Bed': '67be98a8ea3e2aaa233b870d',
 'ECG Machine': '67be98a8ea3e2aaa233b870e',
 'MRI Scanner': '67be98a8ea3e2aaa233b870f',
 'Infusion Pump': '67be98a8ea3e2aaa233b8710',
 'Gloves': '67be98a8ea3e2aaa233b8711',
 'IV Drip': '67be98a9ea3e2aaa233b8712',
 'Surgical Mask': '67be98a9ea3e2aaa233b8713',
 'Gown': '67be98a9ea3e2aaa233b8714',
 'Face Shield': '67be98a9ea3e2aaa233b8715',
 'Syringe': '67be98a9ea3e2aaa233b8716',
 'Bandages': '67be98a9ea3e2aaa233b8717',
 'Blood Bags': '67be98a9ea3e2aaa233b8718',
 'Antiseptic Solution': '67be98aaea3e2aaa233b8719',
 'Cotton Rolls': '67be98aaea3e2aaa233b871a'}

In [164]:
# Update _id column of stock listing
stock_listing_df3['listing'] = stock_listing_df3['title'].map(lambda x: new_item_map[x])
stock_listing_df3.drop('title', axis=1, inplace=True)
stock_listing_df3

,listing,acquisitionDate,status,unitCost,quantity,avgUsagePerDay,supplierName,supplierContact,expirationDate,storageLocation
0,67be98a8ea3e2aaa233b8709,2022-01-01,Available,2628329,22,5,Coleman-Bailey Medical Supplies,+63 906 473 8909,NaN,ICU Equipment Room
1,67be98a8ea3e2aaa233b870b,2022-01-01,Available,650745,6,0,Miles Inc Medical Supplies,+63 951 553 0757,NaN,Emergency Equipment Storage
2,67be98a8ea3e2aaa233b870c,2022-01-01,Available,15486,23,0,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Ward Equipment Room
3,67be98a8ea3e2aaa233b870e,2022-01-01,Available,314121,2,2,Wang-Foster Medical Supplies,+63 922 748 0407,NaN,Cardiology Equipment Room
4,67be98a8ea3e2aaa233b870f,2022-01-01,Available,40522506,1,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,Radiology Imaging Suite
...,...,...,...,...,...,...,...,...,...,...
18643,67be98a9ea3e2aaa233b8716,2024-12-31,Available,20,5990,680,Coleman-Bailey Medical Supplies,+63 906 473 8909,1.893283e+12,Pharmacy Storage
18644,67be98a8ea3e2aaa233b8709,2024-12-31,Available,1852964,22,1,"Khan, Johnston and Nguyen Medical Supplies",+63 962 472 5531,NaN,ICU Equipment Room
18645,67be98a9ea3e2aaa233b8717,2024-12-31,Available,11,3405,430,Wu-Flores Medical Supplies,+63 973 346 5410,1.893283e+12,Medical Supply Room
18646,67be98a8ea3e2aaa233b8711,2024-12-31,Available,13,4101,708,Miles Inc Medical Supplies,+63 951 553 0757,1.893283e+12,Medical Supply Room


In [165]:
# Export stock_listing_v3.json (to export to database)
if not os.path.exists('./data/stock_listing_v3.json'):
    stock_listing_df1.to_json('./data/stock_listing_v3.json', orient='records')
    print('File saved successfully!')
else:
    print('File already exists!')


File saved successfully!
